<a href="https://colab.research.google.com/github/riya-chougule/Image_segmentation_for_self_driving_cars_using_deep_learning_techniques/blob/main/model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pathlib import Path
from fastai.vision import *
import wandb
from wandb.fastai import WandbCallback
from functools import partialmethod

In [2]:
# Initialize W&B project
wandb.init(project="semantic-segmentation")

# Define hyper-parameters
config = wandb.config           # for shortening
config.framework = "fast.ai"    
config.img_size = (360, 640)    # dimensions of resized image 
config.batch_size = 4           # Batch size during training
config.epochs = 20              # Number of epochs for training
encoder = models.resnet18       # encoder of unet (contracting path)
config.encoder = encoder.__name__
config.pretrained = True        # whether we use a frozen pre-trained encoder
config.weight_decay = 1e-2     # weight decay applied on layers
config.bn_weight_decay = False  # whether weight decay is applied on batch norm layers
config.one_cycle = True         
config.learning_rate = 3e-3     # learning rate
save_model = True

wandb: Currently logged in as: riya-chougule (use `wandb login --relogin` to force relogin)


In [3]:
# Custom values to filter runs
config.training_stages = 2

# Data paths
path_data = Path('/content/drive/MyDrive/Kaggle/bdd100k_seg/bdd100k/seg')
path_lbl = path_data / 'labels'
path_img = path_data / 'images'

# Associate a label to an input
get_y_fn = lambda x: path_lbl / x.parts[-2] / f'{x.stem}_train_id.png'

In [4]:
# Segmentation Classes extracted from dataset source code
# See https://github.com/ucbdrive/bdd-data/blob/master/bdd_data/label.py
segmentation_classes = [
    'road', 'sidewalk', 'building', 'wall', 'fence', 'pole', 'traffic light',
    'traffic sign', 'vegetation', 'terrain', 'sky', 'person', 'rider', 'car',
    'truck', 'bus', 'train', 'motorcycle', 'bicycle', 'void'
]
void_code = 19  # used to define accuracy and disconsider unlabeled pixels

In [5]:
# Load data into train & validation sets
src = (SegmentationItemList.from_folder(path_img).use_partial_data(0.01)
       .split_by_folder(train='train', valid='val')
       .label_from_func(get_y_fn, classes=segmentation_classes))

# Resize, augment, load in batch & normalize (so we can use pre-trained networks)
data = (src.transform(get_transforms(), size=config.img_size, tfm_y=True)
        .databunch(bs=config.batch_size)
        .normalize(imagenet_stats))

In [6]:
# Define accuracy & ignore unlabeled pixels
def acc(input, target):
    target = target.squeeze(1)
    mask = target != void_code
    return (input.argmax(dim=1)[mask] == target[mask]).float().mean()

In [7]:
# Create NN
learn = unet_learner(
    data,
    arch=encoder,
    pretrained=config.pretrained,
    metrics=acc,
    wd=config.weight_decay,
    bn_wd=config.bn_weight_decay,
    callback_fns=partial(WandbCallback, save_model=save_model, monitor='acc', input_type='images'))

In [8]:
# Train
if config.one_cycle:
    learn.fit_one_cycle(
        config.epochs // 2,
        max_lr=slice(config.learning_rate))
    learn.unfreeze()
    learn.fit_one_cycle(
        config.epochs // 2,
        max_lr=slice(config.learning_rate / 100,
                     config.learning_rate / 10))
else:
    learn.fit(
        config.epochs,
        lr=slice(config.learning_rate))

epoch,train_loss,valid_loss,acc,time
0,2.054692,1.224326,0.669409,00:23
1,1.992331,1.571111,0.355714,00:12
2,2.057285,1.664393,0.436778,00:12
3,2.165433,1.609397,0.508266,00:12
4,2.042537,1.426775,0.433439,00:12
5,1.890583,1.244051,0.564548,00:12
6,1.749517,1.154886,0.563719,00:12
7,1.599788,1.051275,0.627891,00:12
8,1.463508,1.074189,0.611595,00:12
9,1.370301,1.055574,0.625905,00:12


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:795: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


Better model found at epoch 0 with acc value: 0.6694086194038391.
Loaded best saved model from /content/wandb/run-20210318_192616-1bon2gu3/files/bestmodel.pth


epoch,train_loss,valid_loss,acc,time
0,1.400255,0.982005,0.674887,00:13
1,1.341155,0.941101,0.738613,00:13
2,1.274747,0.786095,0.760430,00:13
3,1.167290,0.685745,0.833505,00:13
4,1.047046,0.611362,0.839518,00:13
5,0.976607,0.606744,0.845238,00:13
6,0.886535,0.478893,0.876563,00:13
7,0.827535,0.492752,0.872245,00:13
8,0.764293,0.438786,0.886581,00:13
9,0.722190,0.439871,0.885276,00:13


Better model found at epoch 0 with acc value: 0.6748870611190796.
Better model found at epoch 1 with acc value: 0.7386133670806885.
Better model found at epoch 2 with acc value: 0.7604303956031799.
Better model found at epoch 3 with acc value: 0.8335053324699402.
Better model found at epoch 4 with acc value: 0.8395175933837891.
Better model found at epoch 5 with acc value: 0.8452381491661072.
Better model found at epoch 6 with acc value: 0.8765628933906555.
Better model found at epoch 8 with acc value: 0.8865813612937927.
Loaded best saved model from /content/wandb/run-20210318_192616-1bon2gu3/files/bestmodel.pth
